# 1. 감정분류_BERT

## (1) import

In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm.notebook import tqdm
import random
import time

# 전처리
from sklearn.preprocessing import LabelEncoder

# 학습
from sklearn.model_selection import train_test_split

import tensorflow as tf

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

import keras
from keras.utils import pad_sequences

## (2) 데이터 확인

In [1]:
folder_path = ''

In [ ]:
train = pd.read_csv(folder_path + 'train.csv')
test = pd.read_csv(folder_path + 'test.csv')

In [ ]:
display(train.head())
display(test.head())

In [ ]:
## 선택: 컬럼명 변경 ##
train = train.rename(columns={'감정': 'label', '문장': 'test'})
test = test.rename(columns={'감정': 'label', '문장': 'test'})

## (3) 데이터 전처리

### 1. 레이블링 ( 문자 -> 숫자로 인코딩 )

In [ ]:
encoder = LabelEncoder()
train['label'] = encoder.fit_transform(train['label'])
# 라벨이 정답이라, test 데이터에는 없음 -> train 데이터만 fit_transform

### 2. 중복제거

In [ ]:
# text 열 중복 제거
train.drop_duplicates(subset=['text'], inplace=True)
test.drop_duplicates(subset=['text'], inplace=True)

### 3. 특수문자 제거 ( 정규표현식 )

In [ ]:
train['text'] = train['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")
test['text'] = test['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")

### 4. Null 값 제거

In [ ]:
# 하나라도 결측값이 있는 행 또는 열 제거
train = train.dropna(how='any')
test = test.dropna(how='any')

## (4) 학습

In [ ]:
# 훈련 데이터 비율 지정
rate = 0.8

### 1. BERT 형식으로 변환 ( [CLS] 문장 [SEP] )

In [ ]:
# 시작과 끝에 특수 토큰을 추가하여 문장의 시작과 끝을 알림

sentences = train['text']

sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 추출 ( test 데이터에는 없음 )
labels = train['label'].values

### 2, BERT로 토큰화하기

In [ ]:
bert_model = "bert-base-multilingual-cased"

# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

### 3. 문장 길이 조회

In [ ]:
# 문장 길이 전체 그래프로 문장의 전체 분포 파악 가능
plt.hist([len(s) for s in sentences], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 길이가 n 이하인 데이터의 비율 계산 함수
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if len(s) <= max_len:
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s' % (max_len, (cnt / len(nested_list)) * 100))

max_len = 128
below_threshold_len(max_len, sentences) # 이 함수 보고 max_len 설정

### 4. 토큰 인덱스 변환, 패딩 및 어텐션 마스크 세팅

In [ ]:
# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 max_len 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")

# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

### 5. 훈련데이터와 검증데이터 분리 및 배치 세팅

In [ ]:
batch_size = 64

# 훈련셋과 검증셋으로 분리
validation_size = 0.2

random_state = 1

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=random_state,
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=random_state,
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### 6. GPU 세팅
- Colab에서만 진행해야 할 지, 시험환경에서도 진행해야 할 지 의문

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print("Found GPU at: {}".format(device_name))
else:
    raise SystemError("GPU device not found")

# 디바이스 설정
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

#### 7. 딥러닝 모델 세팅

In [ ]:
# 분류를 위한 BERT 모델 생성

# 분류개수 ( 변동 가능 )
classification_count = 6

# 반복 횟수 ( 변동 가능 )
epochs = 3

model = BertForSequenceClassification.from_pretrained(bert_model, num_labels=classification_count)
model.cuda()

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr=2e-5, # 학습률
                  eps=1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 총 훈련 스탭 : 배치반복 횟수 * 에폭수
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

#### 8. 딥러닝 모델 간략하게 보기

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))s

#### 9 딥러닝 학습

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed))) # elapsed에 왜 괄호를 한 번 더 하는지.. 의문.. 안 되면 빼고 해보기

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

# 학습
## 현재 150000개의 데이터로는 1 epoch 50분 정도의 시간이 소요됩니다.
## 4번의 epoch이기에 약 200분, 3시간이 넘는 학습 시간이 필요합니다. -> 3번으로 줄이신 듯
## 연습을 하실 때에는 1000개 정도로 데이터량을 줄여서 테스트 해보실 것을 권장드립니다.

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    print("")
    print("======= Epoch {:} / {:} =======".format(epoch_i + 1, epochs))
    print("Training...")

    t0 = time.time()

    total_train_loss = 0

    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시 / 매 40 배치마다
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 그래디언트 초기화
        model.zero_grad()

        output = model(b_input_ids,
                       token_type_ids=None,
                       attention_mask=b_input_mask,
                       labels=b_labels)
        
        # 총 로스 계산
        loss = output[0]
        total_train_loss += loss.item()
        logits = output[1]

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_train_loss / len(train_dataloader)

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    print("")
    print("Running Validation...")

    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 그래디언트 계산 안 함
        with torch.no_grad():
            output = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels)
        
        # 로스 구함
        loss = output.loss
        total_eval_loss += loss.item()
        logits = output.logits

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training. Accur': avg_val_accuracy,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))

#### 10. 딥러닝 학습 간단 보기

In [ ]:
pd.set_option('display.max_columns', 100)

df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')

print(df_stats.to_string())

sns.set(style="darkgrid")

sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12, 6)

plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([i for i in range(1, epochs + 1)])

plt.show()

## (5) 결과 제출

In [ ]:
temp_sentence = '무릎이 아파서 잠시 걷는 것조차 힘들어졌어. 내가 앞으로 무얼 할 수 있을까?'

# 입력 데이터 변환 함수
def convert_input_data(sentences):
    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

logits = test_sentences([temp_sentence])

print(logits)
print(np.argmax(logits))

In [ ]:
# 테스트 데이터를 for문으로 test_sentences 함수에 하나씩 넣어 감정 분류 해주고,
# 숫자로 되어 있는 것을 다시 디코딩 작업하여 답안지 생성.
result = []
x_test = test['text']

for item in x_test:
    logits = test_sentences([item])
    pred = np.argmax(logits)
    pred = encoder.inverse_transform([pred])[0]
    result.append(pred)

print(result)

In [ ]:
test['result'] = result

In [ ]:
test.to_csv('01073002902_2.csv', index=False)
torch.save(model, "01073002902_2.h5")